In [1]:
import pandas as pd
import yfinance as yf

In [2]:
df = yf.download("AAPL", period="60D", interval="15m")

[*********************100%***********************]  1 of 1 completed


In [3]:
df = df.tz_localize(None)

In [18]:
df["growing"] = df["Open"].gt(df["Open"].shift())
df["streak_start"] = df["growing"].ne(df["growing"].shift())
df["streak_no"] = df["streak_start"].cumsum()
df["streak_count"] = df.groupby("streak_no").cumcount().add(1)

colour_map = growing_streaks.values 

df.iloc[:15,:].style.apply(lambda x: ["background: green" 
                          if colour_map[i] else "background:red" 
                          for i,v in enumerate(x)], 
            axis=0, subset=["Open"])



,Open,High,Low,Close,Adj Close,Volume,streak_no,streak_length,growing_streaks,streak_start,growing_streak,growing,streak_count
Datetime,,,,,,,,,,,,,
2021-09-21 09:30:00,143.929993,144.110001,143.210098,143.809998,143.809998,8763922,1,1,False,True,False,False,1
2021-09-21 09:45:00,143.839996,144.600006,143.839996,144.117294,144.117294,6309348,1,2,False,False,False,False,2
2021-09-21 10:00:00,144.110001,144.300003,143.380005,143.641998,143.641998,4298166,2,1,True,True,True,True,1
2021-09-21 10:15:00,143.639999,143.929993,143.419998,143.789993,143.789993,3529039,3,1,False,True,False,False,1
2021-09-21 10:30:00,143.785004,143.869904,143.350006,143.589996,143.589996,2487220,4,1,True,True,True,True,1
2021-09-21 10:45:00,143.619995,143.690002,142.889999,143.014999,143.014999,3567569,5,1,False,True,False,False,1
2021-09-21 11:00:00,143.020004,143.822800,142.779999,143.820007,143.820007,3040861,5,2,False,False,False,False,2
2021-09-21 11:15:00,143.820007,143.899994,143.419998,143.520004,143.520004,2281219,6,1,True,True,True,True,1
2021-09-21 11:30:00,143.520004,144.199997,143.520004,144.113693,144.113693,2685890,7,1,False,True,False,False,1


In [19]:
df.loc[:df.index[10], ["streak_start", "streak_no", "streak_count"]]

,streak_start,streak_no,streak_count
Datetime,,,
2021-09-21 09:30:00,True,1,1
2021-09-21 09:45:00,False,1,2
2021-09-21 10:00:00,True,2,1
2021-09-21 10:15:00,True,3,1
2021-09-21 10:30:00,True,4,1
2021-09-21 10:45:00,True,5,1
2021-09-21 11:00:00,False,5,2
2021-09-21 11:15:00,True,6,1
2021-09-21 11:30:00,True,7,1


In [21]:
max_streak_length = df.loc[df["streak_count"].idxmax(), "streak_count"]

In [31]:
df.loc[:df["streak_count"].idxmax(), ["Open", "streak_no", "streak_count"]]\
  .tail(max_streak_length).style.apply(lambda x: ["background:red" for v in x], axis=0, subset=["Open"])

,Open,streak_no,streak_count
Datetime,,,
2021-10-12 14:45:00,142.125504,213,1
2021-10-12 15:00:00,141.970001,213,2
2021-10-12 15:15:00,141.957001,213,3
2021-10-12 15:30:00,141.809998,213,4
2021-10-12 15:45:00,141.494995,213,5
2021-10-13 09:30:00,141.235001,213,6
2021-10-13 09:45:00,140.160004,213,7
2021-10-13 10:00:00,139.729996,213,8
2021-10-13 10:15:00,139.589996,213,9


In [70]:
df.reset_index().loc[:, ["Datetime", "Open","streak_no"]]\
  .groupby("streak_no")\
  .agg(
    start = pd.NamedAgg(column="Datetime", aggfunc="first"),
    end = pd.NamedAgg(column="Datetime", aggfunc="last"),
    growth = pd.NamedAgg(column="Open", aggfunc=(lambda x: (x.iloc[-1]-x.iloc[0])/x.iloc[0])),
    size = pd.NamedAgg(column="Open", aggfunc="size"),
).sort_values(by="size", ascending=False).iloc[:10,:].style.apply(lambda x: ["background:red" if v<0 else "background:green" for v in x], axis=0, subset=["growth"])

,start,end,growth,size
streak_no,,,,
213,2021-10-12 14:45:00,2021-10-13 10:15:00,-0.017840,9
111,2021-09-30 15:30:00,2021-10-01 10:45:00,-0.024251,8
306,2021-10-21 14:30:00,2021-10-22 09:45:00,0.008742,8
726,2021-12-07 15:00:00,2021-12-08 10:15:00,0.023127,8
527,2021-11-15 10:00:00,2021-11-15 11:45:00,-0.009473,8
696,2021-12-03 15:15:00,2021-12-06 10:15:00,0.043785,7
409,2021-11-02 13:15:00,2021-11-02 14:45:00,-0.007283,7
596,2021-11-19 15:30:00,2021-11-22 10:30:00,0.030051,7
531,2021-11-15 13:00:00,2021-11-15 14:30:00,-0.001100,7
